In [18]:
import torch
import pandas as pd


In [19]:
{
  "request_id": "string",
  "created_at": "datetime",
  "customer_message": "string",
  "intent": "string",
  "target": "string",
  "timeframe": "string",
  "status": "string",
  "department": "string",
  "assignee": "string"
}


{'request_id': 'string',
 'created_at': 'datetime',
 'customer_message': 'string',
 'intent': 'string',
 'target': 'string',
 'timeframe': 'string',
 'status': 'string',
 'department': 'string',
 'assignee': 'string'}

In [20]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json

# Load FLAN-T5
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def extract_fields(text):
    prompt = f"""
### Task
You are an AI assistant that helps a support team extract structured information from customer messages.

### Instruction
From the message below, extract:
- `intent`: what the customer is trying to do
- `target`: what the request is about
- `timeframe`: any date, urgency or time expression mentioned


### Examples
Input: I need to reschedule my pantry visit from Saturday to next week.
Output: {"intent": "reschedule appointment", "target": "pantry visit", "timeframe": "next week"}

Input: Can someone help me submit my SNAP renewal today?
Output: {"intent": "submit renewal", "target": "SNAP", "timeframe": "today"}

### Message
{text.strip()}

### Message
{text.strip()}

### Output
Respond only in valid JSON format:
{{"intent": "...", "target": "...", "timeframe": "..."}}
""".strip()

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    raw = tokenizer.decode(output[0], skip_special_tokens=True)

    try:
        fields = json.loads(raw)
    except json.JSONDecodeError:
        fields = {"intent": "", "target": "", "timeframe": ""}

    return fields


In [22]:
USERNAME="samarthsingh"
USER_PATH = '/Users/{}/Downloads/'.format(USERNAME)
df=pd.read_csv(USER_PATH+'data_case2.xlsx - PSUP_Jira_Data_Email_Scrambled.csv')

json_records = df.apply(build_ticket_json, axis=1).tolist()
json_records

KeyError: 'customer_message'

In [2]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# 
# # Load FLAN-T5 base model
# model_name = "google/flan-t5-small"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)
# 
# def generate_summary(text):
#     prompt = f"Summarize this customer request in one sentence: {text.strip()}"
#     input_ids = tokenizer(prompt, return_tensors="pt").input_ids
#     output = model.generate(input_ids, max_length=40, num_beams=4, early_stopping=True)
#     summary = tokenizer.decode(output[0], skip_special_tokens=True)
#     return summary
# 
# test_text = "Hi, I just moved here and I’m trying to get assistance for my family. Is there a way to schedule a pantry visit this weekend?"
# print("Summary:", generate_summary(test_text))


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Summary: Hi, I just moved here and I’m trying to get assistance for my family. Is there a way to schedule a pantry visit this weekend?


In [ ]:
{
  "request_id": "IT-7834",
  "created_at": "2025-03-25T12:08:00",
  "intent": "Schedule Appointment",
  "target": "Pantry Visit",
  "timeframe": "Next Week",
  "customer_message": "Hi, I need to schedule a pantry appointment for next week. Can you help?",
  "department": "Pantry Services",
  "status": "Open",
  "assignee": ""
}
